In [7]:
from model.ffnn import FFNN

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, accuracy_score
import time, os

MODEL_FILENAME = "ffnn_model.pkl"

In [8]:
train_samples = 5000

# Inisialisasi model
model = FFNN(
    jumlah_neuron=[784, 128, 64, 64, 10],
    fungsi_aktivasi=["ReLU", "ReLU", "ReLU", "Softmax"],
    fungsi_loss="CategoricalCrossEntropy",
    inisialisasi_bobot="xavier-normal",
    seed=123123,
    lower_bound=-1,
    upper_bound=1,
    mean=0,
    std=1,
    verbose=1,
    l1_lambda=0.0,  # Tidak menggunakan L1 regularization
    l2_lambda=0.01,
)

# Cek apakah model sudah ada
if os.path.exists(MODEL_FILENAME):
    model = FFNN.load_model_pickle(model, MODEL_FILENAME) # Load model dari file
    train_needed = False
else:
    train_needed = True

In [9]:
# Train or load

# Load MNIST dataset
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X = X / 255.0

# Split dataset into training, validation, and test sets
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=10000, random_state=123123
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, train_size=5000, test_size=1000, random_state=42
)

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))
y_val = encoder.transform(y_val.reshape(-1, 1))

if train_needed:
    # Catat waktu mulai training
    start_time = time.time()

    # Training model
    model.fit(X=X_train, y=y_train, batch=100, lr=0.1, epochs=500, X_val=X_val, y_val=y_val)

    # Catat waktu selesai training
    end_time = time.time()
    print(f"Training time: {end_time - start_time:.2f} seconds")

    # Simpan model setelah training
    model.save_model_pickle(MODEL_FILENAME)

[██████████████████████████████████████████████████] - 500/500 - 100.0% - 103.80s - Training Loss: 0.655820 - Val Loss: 0.693733
Final Training Loss: 0.655820
Final Validation Loss: 0.693733
Training time: 103.80 seconds
Model berhasil disimpan ke ffnn_model.pkl


In [10]:
# Prediksi
pred = model.predict(X_test)

# Konversi y_test dari one-hot encoding ke label asli
y_test_labels = np.argmax(y_test, axis=1)

# Hitung F1-score
model_f1 = f1_score(y_test_labels, pred, average="macro")
print(f"Model F1-Score: {model_f1:.4f}")

# Hitung Accuracy
model_accuracy = accuracy_score(y_test_labels, pred)
print(f"Model Accuracy: {model_accuracy:.4f}")

Model F1-Score: 0.7963
Model Accuracy: 0.8007


In [11]:
# Loop GUI dengan perintah ini
%gui qt

In [12]:
# Visualize model
gui = model.model_visualize()
gui.show()

[LossDistribution] Title: Training Loss
[LossDistribution] Loss values (500): [np.float64(2.2234814977758783), np.float64(2.1457940183372783), np.float64(2.076044628005137), np.float64(2.002592963844335), np.float64(1.920328231022935), np.float64(1.8304763134618385), np.float64(1.7371871478926215), np.float64(1.6414712603532535), np.float64(1.5433527166793597), np.float64(1.4461093413870787), np.float64(1.3480415258144665), np.float64(1.2545053516543136), np.float64(1.1676956052282144), np.float64(1.0926953487311493), np.float64(1.0245453241278106), np.float64(0.9664070309801075), np.float64(0.9198765479987459), np.float64(0.8809633263883531), np.float64(0.8471924186418257), np.float64(0.8218101752517075), np.float64(0.7985508071513308), np.float64(0.7812334359313919), np.float64(0.7667032128823396), np.float64(0.7525153215881523), np.float64(0.7428645597992151), np.float64(0.7324398396242172), np.float64(0.7272926909070062), np.float64(0.7196100006045196), np.float64(0.712510656151551